<h1> Sleep Health and Lifestyle </h1>

![Alt Text](Sleep_picture.jpg)

<h2> The Problem </h2>

Living in today's fast-paced world, sleep has become an overlooked components of our health. Poor sleep habits, rising stress levels, and sedantary routines and increasinly linked to various chronic health problems, however the roots of these sleep habits and disorders remained complex and multidimensional.

With this, organization and healthcare providers may need data-driven insights to understand different aspects that contribute to poor sleep, how it differentiates between different demographics, and what interventions are possible that might improve health outcomes.

🔍 
<i>How can we identify and address the key factors influencing poor sleep quality and sleep disorders across different demographics to support healthier living?</i>

<h3> Dataset Overview </h3>

The dataset consists of 400 indviduals and includes the following variables:

- Demographics: Person ID, Age, Gender, Occupation
- Sleep Metrics: Sleep Duration, Quality of Sleep (1-10), Sleep Disorder type
- Health Indicators: BMI Category, Blood Pressure, Heart Rate (bpm)
- Lifestyle Factors: Physical Activity (minutes/day), Stress Level (scale 1-10),